In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [13]:
lptp = pd.read_csv('lptp.csv')
lptp.head()

,name,r_iris_rx,r_iris_ry,r_iris_cx,r_iris_cy,condition,r_iris_point_1_x,r_iris_point_1_y,r_iris_point_2_x,r_iris_point_2_y,...,l_eyelid_pcl_12_x,l_eyelid_pcl_12_y,l_eyelid_pcl_13_x,l_eyelid_pcl_13_y,l_eyelid_pcl_14_x,l_eyelid_pcl_14_y,l_eyelid_pcl_15_x,l_eyelid_pcl_15_y,l_eyelid_pcl_16_x,l_eyelid_pcl_16_y
0,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_007.png,10.0,8.5,1047.0,433.5,A,1037.0,425.0,1057.0,442.0,...,942.0,434.0,934.0,433.0,926.0,435.0,921.0,436.0,918.0,438.0
1,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_013.png,10.0,8.5,1048.0,433.5,A,1038.0,425.0,1058.0,442.0,...,942.0,434.0,933.0,434.0,925.0,435.0,920.0,437.0,917.0,438.0
2,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_017.png,10.0,9.0,1048.0,434.0,A,1038.0,425.0,1058.0,443.0,...,942.0,434.0,934.0,434.0,926.0,435.0,921.0,437.0,918.0,438.0
3,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_025.png,10.5,8.5,1047.5,434.5,A,1037.0,426.0,1058.0,443.0,...,943.0,435.0,934.0,435.0,926.0,436.0,921.0,438.0,919.0,439.0
4,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_028.png,10.0,8.5,1047.0,434.5,A,1037.0,426.0,1057.0,443.0,...,942.0,435.0,933.0,435.0,925.0,436.0,920.0,438.0,918.0,439.0


In [14]:
lptp.columns

Index(['name', 'r_iris_rx', 'r_iris_ry', 'r_iris_cx', 'r_iris_cy', 'condition',
       'r_iris_point_1_x', 'r_iris_point_1_y', 'r_iris_point_2_x',
       'r_iris_point_2_y', 'l_iris_point_1_x', 'l_iris_point_1_y',
       'l_iris_point_2_x', 'l_iris_point_2_y', 'r_eyelid_pcl_1_x',
       'r_eyelid_pcl_1_y', 'r_eyelid_pcl_2_x', 'r_eyelid_pcl_2_y',
       'r_eyelid_pcl_3_x', 'r_eyelid_pcl_3_y', 'r_eyelid_pcl_4_x',
       'r_eyelid_pcl_4_y', 'r_eyelid_pcl_5_x', 'r_eyelid_pcl_5_y',
       'r_eyelid_pcl_6_x', 'r_eyelid_pcl_6_y', 'r_eyelid_pcl_7_x',
       'r_eyelid_pcl_7_y', 'r_eyelid_pcl_8_x', 'r_eyelid_pcl_8_y',
       'r_eyelid_pcl_9_x', 'r_eyelid_pcl_9_y', 'r_eyelid_pcl_10_x',
       'r_eyelid_pcl_10_y', 'r_eyelid_pcl_11_x', 'r_eyelid_pcl_11_y',
       'r_eyelid_pcl_12_x', 'r_eyelid_pcl_12_y', 'r_eyelid_pcl_13_x',
       'r_eyelid_pcl_13_y', 'r_eyelid_pcl_14_x', 'r_eyelid_pcl_14_y',
       'r_eyelid_pcl_15_x', 'r_eyelid_pcl_15_y', 'r_eyelid_pcl_16_x',
       'r_eyelid_pcl_16_y', 'l_eyeli

In [63]:
## 최대값이 331, 340을 최대 길이로 설정

s = lptp['name']
match = []
for i in s:
    match.append(int(re.findall(r"(\d{3})(?=\.png$)", i)[0]))
print(np.max(match))
print(np.min(match))

331
1


In [66]:
lptp.shape

(62061, 78)

In [67]:
lptp_drop = lptp.dropna()

In [68]:
def insert_zero_padding(data : pd.DataFrame, pad : int):
    result = []
    pre_idx = 0
    for row in range(data.shape[0]):
        tmp = data.iloc[row]
        idx_list = re.findall(r"(\d{3})(?=\.png$)", tmp['name'])
        idx = int(idx_list[0])
        if(idx < pre_idx):
            for i in range(pre_idx+1, 341):
                zero_df = pd.DataFrame([[f'{strstr}{i:03d}.png'] + [pad] * (data.shape[1]-1)], columns=data.columns)
                result.append(zero_df.iloc[0])
            pre_idx = 0
        strstr = re.findall(r"^(.*?)(\d{3}\.png)$", tmp['name'])[0][0]
        for i in range(pre_idx+1, idx):
            zero_df = pd.DataFrame([[f'{strstr}{i:03d}.png'] + [pad] * (data.shape[1]-1)], columns=data.columns)
            result.append(zero_df.iloc[0])
        result.append(tmp)
        pre_idx = idx
    return pd.DataFrame(result).reset_index(drop=True)
        

In [69]:
lptp_pad = insert_zero_padding(lptp_drop, 0)

In [74]:
lptp_pad.to_csv('lptp_pad.csv', index=False)

In [76]:
tablet = pd.read_csv('tablet.csv')
tablet_drop = tablet.dropna()

In [77]:
tablet_pad = insert_zero_padding(tablet_drop, 0)

In [78]:
tablet_pad.head(300)

,name,r_iris_rx,r_iris_ry,r_iris_cx,r_iris_cy,condition,r_iris_point_1_x,r_iris_point_1_y,r_iris_point_2_x,r_iris_point_2_y,...,l_eyelid_pcl_12_x,l_eyelid_pcl_12_y,l_eyelid_pcl_13_x,l_eyelid_pcl_13_y,l_eyelid_pcl_14_x,l_eyelid_pcl_14_y,l_eyelid_pcl_15_x,l_eyelid_pcl_15_y,l_eyelid_pcl_16_x,l_eyelid_pcl_16_y
0,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_001.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_002.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_003.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_004.png,14.5,11.0,549.5,470.0,A,535.0,459.0,564.0,481.0,...,409.0,488.0,398.0,490.0,387.0,494.0,381.0,498.0,378.0,501.0
4,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_005.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_296.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_297.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_298.png,14.5,11.5,567.5,469.5,A,553.0,458.0,582.0,481.0,...,428.0,489.0,417.0,491.0,406.0,495.0,400.0,500.0,397.0,503.0
298,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_299.png,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
tablet_pad.isnull().sum()

name                 0
r_iris_rx            0
r_iris_ry            0
r_iris_cx            0
r_iris_cy            0
                    ..
l_eyelid_pcl_14_y    0
l_eyelid_pcl_15_x    0
l_eyelid_pcl_15_y    0
l_eyelid_pcl_16_x    0
l_eyelid_pcl_16_y    0
Length: 78, dtype: int64

In [80]:
tablet_pad.to_csv('tablet_pad.csv', index=False)